In [85]:
import pandas as pd
import numpy as np


In [86]:
df = pd.read_csv('clean_data.csv', index_col=0)
df

,WEEK OF,STATION,STATION CITY,STATION PROVINCE,STATION LATITUDE,STATION LONGITUDE,CHART POSITION,ARTIST NAME(S),ARTIST COUNTRY,ARTIST HOME CITY,...,LABEL TYPE,LANGUAGE OF MUSIC,VISIBLE ETHNIC MINORITY,CENSUS RACE CLASSIFICATION,ARTIST GENDER,M,A,P,L,CANADIAN CONTENT
0,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,4.0,breakestra,us,"los angeles, ca",...,indie,english,yes,mixed group,male group,no,no,no,no,no
1,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,5.0,candy's .22,us,"los angeles, ca",...,indie,english,no,white,male group,no,no,no,no,no
2,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,7.0,dangerdoom,us,"new york, ny",...,indie,english,yes,black,male,no,no,no,no,no
3,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,8.0,blockhead,us,"new york, ny",...,indie,english,no,white,male,no,no,no,no,no
4,2006-01-10,cjsr,edmonton,ab,53.55,-113.50,9.0,blackalicious,us,"sacramento, ca",...,indie,english,yes,black,male group,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35854,2010-03-02,ckut,montreal,qc,45.50,-73.58,4.0,kae sun,cdn,"toronto, on",...,indie,english,yes,black,male,no,no,no,no,no
35855,2010-03-02,ckut,montreal,qc,45.50,-73.58,5.0,rel!g!on,cdn,"vancouver, bc",...,indie,english,yes,black,male,yes,yes,yes,yes,yes
35856,2010-03-02,ckut,montreal,qc,45.50,-73.58,6.0,swollen members,cdn,"vancouver, bc",...,indie,english,yes,mixed group,male group,yes,yes,yes,yes,yes
35857,2010-03-02,ckut,montreal,qc,45.50,-73.58,7.0,chokeules,cdn,"toronto, on",...,indie,english,no,white,male,yes,yes,yes,yes,yes


In [148]:
def expand_column_to_percentiles(column:str, group:str, dataframe:pd.DataFrame, origin:pd.DataFrame):
    for x in sorted(df[column].unique()):
        x_plays = origin[df[column]==x ].groupby(group)[column].count()
        total_plays = origin.groupby(group)[column].count()
        if type(x) == np.float64:
            x = int(x)
        dataframe[f'{column} {x}'] = (x_plays / total_plays)

In [149]:
artist_data = df.groupby('ARTIST NAME(S)').agg(lambda x: x.value_counts().index[0])[
                                                [ 'ARTIST HOME LATITUDE',
                                                  'ARTIST HOME LONGITUDE',
                                                  'ARTIST HOME CITY',
                                                  'ARTIST COUNTRY',
                                                  'LANGUAGE OF MUSIC',
                                                  'VISIBLE ETHNIC MINORITY',
                                                  'CENSUS RACE CLASSIFICATION',
                                                  'ARTIST GENDER',
                                                  'CANADIAN CONTENT']
                                                ]
artist_data['ALBUMS PLAYED'] =  df.groupby('ARTIST NAME(S)')['ALBUM NAME'].nunique()
artist_data['STATIONS PLAYED ON'] =  df.groupby('ARTIST NAME(S)')['STATION'].nunique()
artist_data['MEAN CHART POSITION'] =  df.groupby('ARTIST NAME(S)')['CHART POSITION'].mean().round().astype(int)
artist_data['TOTAL DAYS PLAYED'] =  df.groupby('ARTIST NAME(S)')['ALBUM NAME'].count()

expand_column_to_percentiles('STATION', 'ARTIST NAME(S)', artist_data, df)
expand_column_to_percentiles('CHART POSITION', 'ARTIST NAME(S)', artist_data, df)

artist_data.replace(np.nan,0, inplace=True)

In [151]:
artist_data.to_csv('artist_table.csv')

In [122]:
def average_position_by_demographic(demographic:str):
    results = {}
    for x in artist_data[demographic].unique():
        average_position = artist_data[artist_data[demographic] == x]['MEAN CHART POSITION'].mean().round(2)
        results[x] = average_position  
    for item in sorted(results.items(), key=lambda x:x[1]):
        print(item)

In [123]:
average_position_by_demographic('ARTIST COUNTRY')
   

('cdn', 5.75)
('uk', 5.89)
('int', 6.21)
('us', 6.22)
('unknown', 6.41)
('us/cdn', 8.0)


In [12]:
average_position_by_demographic('LANGUAGE OF MUSIC')
    
    

('english/arabic', 3.0)
('punjabi', 4.0)
('english/patois', 4.0)
('portuguese', 5.0)
('creole', 5.0)
('other', 5.5)
('unknown', 5.77)
('basque', 6.0)
('yes', 6.0)
('english/french', 6.0)
('english', 6.06)
('french', 6.36)
('spanish', 6.5)
('multi', 7.0)
('english/zulu', 8.0)
('german', 8.5)


In [13]:
average_position_by_demographic('VISIBLE ETHNIC MINORITY')

('yes', 6.04)
('no', 6.09)
('unknown', 7.73)


In [14]:
average_position_by_demographic('CENSUS RACE CLASSIFICATION')

('other asian', 4.0)
('metis', 5.0)
('jewish', 5.0)
('asian indian', 5.55)
('asian', 5.9)
('hispanic', 5.91)
('native canadian', 5.92)
('black', 6.0)
('inuit', 6.0)
('mixed group', 6.06)
('white', 6.09)
('native american', 6.33)
('asian other', 6.6)
('unidentified', 6.67)
('middle eastern', 6.79)
('indian', 7.0)
('indian asian', 7.33)
('east asian', 7.5)
('unknown', 7.73)
('romany', 8.0)


In [15]:
average_position_by_demographic('ARTIST GENDER')

('mixed group', 5.68)
('female', 5.7)
('male', 6.07)
('male group', 6.15)
('female group', 6.67)
('unknown', 7.2)


In [130]:
df['ARTIST COUNTRY'].unique()

array(['us', 'uk', 'int', 'cdn', 'us/cdn', 'unknown'], dtype=object)

In [152]:
station_data = df.groupby('STATION').agg(lambda x: x.value_counts().index[0])[
                                                [ 'STATION LATITUDE',
                                                  'STATION LONGITUDE',
                                                  'STATION CITY',
                                                  'STATION PROVINCE',
                                                  'ARTIST HOME CITY',
                                                  'ARTIST COUNTRY',
                                                  'LABEL TYPE',
                                                  'LABEL NAME'
                                                  ]
                                                ]

expand_column_to_percentiles('ARTIST GENDER', 'STATION', station_data, df)
expand_column_to_percentiles('CENSUS RACE CLASSIFICATION', 'STATION', station_data, df)
expand_column_to_percentiles('ARTIST COUNTRY', 'STATION', station_data, df)
expand_column_to_percentiles('CANADIAN CONTENT', 'STATION', station_data, df)
station_data.rename(columns={ 'ARTIST HOME CITY' : 'MOST PLAYED ARTIST CITY',
                              'LABEL TYPE' : 'MOST PLAYED LABEL TYPE',
                              'LABEL NAME' : 'MOST PLAYED LABEL ',
                              'yes' : 'CAN CON', 
                              'no' : 'NOT CAN CON'

                             }, inplace=True)

station_data.replace(np.nan,0, inplace=True)


In [153]:
station_data.to_csv('artist_table.csv')